In [1]:
import re
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/katiegarrett/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
top_2016 = pd.read_csv('../data/top_2016.csv')
top_2016.head()

,Unnamed: 0,author,author_created_utc,author_flair_css_class,author_flair_text,author_fullname,created_utc,domain,edited,full_link,...,post_hint,preview,contest_mode,spoiler,link_flair_css_class,link_flair_text,distinguished,media,secure_media,date
0,0,Augustane,1.324721e+09,NaN,9th & 11th ELA,t2_6i6f9,1471965288,self.Teachers,1.472499e+09,https://www.reddit.com/r/Teachers/comments/4z6...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-08-23 15:14:48+00:00
1,1,PhantomandaRose,1.409513e+09,NaN,English 9-10,t2_i520j,1470103158,self.Teachers,1.470666e+09,https://www.reddit.com/r/Teachers/comments/4vq...,...,self,{'images': [{'id': 'jeLWXJmwEhuQ6Hv7CrFjKhlcAB...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-08-02 01:59:18+00:00
2,2,supertizer,1.285323e+09,NaN,NaN,t2_4dd3n,1480861135,supertizer.org,NaN,https://www.reddit.com/r/Teachers/comments/5gf...,...,link,{'images': [{'id': 'UbHZzeS-8jeml7jeNZpltwExgN...,False,False,NaN,NaN,NaN,NaN,NaN,2016-12-04 14:18:55+00:00
3,3,supertizer,1.285323e+09,NaN,NaN,t2_4dd3n,1480263081,supertizer.org,NaN,https://www.reddit.com/r/Teachers/comments/5f5...,...,link,{'images': [{'id': 'U52eS2tNRS6V_LFBP-VaNBcN3D...,False,False,NaN,NaN,NaN,NaN,NaN,2016-11-27 16:11:21+00:00
4,4,Namnagort,1.371766e+09,NaN,NaN,t2_c3vjf,1482753867,self.Teachers,NaN,https://www.reddit.com/r/Teachers/comments/5kd...,...,NaN,NaN,False,False,NaN,NaN,NaN,NaN,NaN,2016-12-26 12:04:27+00:00


In [3]:
top_2016["combined"] = top_2016["title"].astype(str) + top_2016["selftext"].astype(str)

In [4]:
top_2016["row_id"] = top_2016.index + 1 
print (top_2016.head(10))

   Unnamed: 0              author  author_created_utc  author_flair_css_class  \
0           0           Augustane        1.324721e+09                     NaN   
1           1     PhantomandaRose        1.409513e+09                     NaN   
2           2          supertizer        1.285323e+09                     NaN   
3           3          supertizer        1.285323e+09                     NaN   
4           4           Namnagort        1.371766e+09                     NaN   
5           5     ragesinggoddess        1.391173e+09                     NaN   
6           6         Lasty_girly        1.448268e+09                     NaN   
7           7          supertizer        1.285323e+09                     NaN   
8           8  Lover_Of_The_Light        1.356850e+09                     NaN   
9           9         drjtelliott        1.479423e+09                     NaN   

  author_flair_text author_fullname  created_utc          domain  \
0    9th & 11th ELA        t2_6i6f9   14

In [5]:
top_2016_subset = top_2016[['row_id', 'combined']].copy()
#data clean-up
#remove all non-aphabet characters
top_2016_subset['combined'] = top_2016_subset['combined'].str.replace("[^a-zA-Z#]", " ")
#covert to lower-case
top_2016_subset['combined'] = top_2016_subset['combined'].str.casefold()
print (top_2016_subset.head(10))

   row_id                                           combined
0       1  i pretended to be a student on the first day  ...
1       2  boycott reddit gifts for teachers   donorschoo...
2       3  this is the pop quiz i give students after tea...
3       4                           the girls are bigger nan
4       5  the real ten tips to avoiding burnouts   drink...
5       6  update  i was asked to resignpreviously  i was...
6       7  i teach english as a second language    one as...
7       8  i ran into a student a couple blocks from scho...
8       9  how to be a good teacherdon t teach to the tes...
9      10  parents are as responsible as teachers for stu...


/var/folders/4_/948td82x7jlf4pz0c4dtqn_00000gn/T/ipykernel_2320/797054238.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  top_2016_subset['combined'] = top_2016_subset['combined'].str.replace("[^a-zA-Z#]", " ")


In [6]:
df1=pd.DataFrame()
df1['row_id']=['99999999999']
df1['sentiment_type']='NA999NA'
df1['sentiment_score']=0

In [7]:
print('Processing sentiment analysis...')
sid = SentimentIntensityAnalyzer()
t_df = df1
for index, row in top_2016_subset.iterrows():
    scores = sid.polarity_scores(row[1])
    for key, value in scores.items():
        temp = [key,value,row[0]]
        df1['row_id']=row[0]
        df1['sentiment_type']=key
        df1['sentiment_score']=value
        t_df=t_df.append(df1)
#remove dummy row with row_id = 99999999999
t_df_cleaned = t_df[t_df.row_id != '99999999999']
#remove duplicates if any exist
t_df_cleaned = t_df_cleaned.drop_duplicates()
# only keep rows where sentiment_type = compound
t_df_cleaned = t_df[t_df.sentiment_type == 'compound']
print(t_df_cleaned.head(10))

Processing sentiment analysis...
   row_id sentiment_type  sentiment_score
0       1       compound           0.9716
0       2       compound           0.8845
0       3       compound           0.0000
0       4       compound           0.0000
0       5       compound          -0.6664
0       6       compound           0.9824
0       7       compound           0.0000
0       8       compound          -0.2263
0       9       compound           0.9869
0      10       compound           0.3182


In [8]:
df_output = pd.merge(top_2016, t_df_cleaned, on='row_id', how='inner')
print(df_output.head(10))

   Unnamed: 0              author  author_created_utc  author_flair_css_class  \
0           0           Augustane        1.324721e+09                     NaN   
1           1     PhantomandaRose        1.409513e+09                     NaN   
2           2          supertizer        1.285323e+09                     NaN   
3           3          supertizer        1.285323e+09                     NaN   
4           4           Namnagort        1.371766e+09                     NaN   
5           5     ragesinggoddess        1.391173e+09                     NaN   
6           6         Lasty_girly        1.448268e+09                     NaN   
7           7          supertizer        1.285323e+09                     NaN   
8           8  Lover_Of_The_Light        1.356850e+09                     NaN   
9           9         drjtelliott        1.479423e+09                     NaN   

  author_flair_text author_fullname  created_utc          domain  \
0    9th & 11th ELA        t2_6i6f9   14

In [9]:
df_output[["sentiment_score"]].describe()

,sentiment_score
count,601.000000
mean,0.305793
std,0.622855
min,-0.997300
25%,-0.038000
50%,0.440400
75%,0.909000
max,0.999500


In [10]:
top_2016_final = df_output

In [11]:
top_2016_final['year'] = 2016

In [12]:
top_2016_final.head()

,Unnamed: 0,author,author_created_utc,author_flair_css_class,author_flair_text,author_fullname,created_utc,domain,edited,full_link,...,link_flair_text,distinguished,media,secure_media,date,combined,row_id,sentiment_type,sentiment_score,year
0,0,Augustane,1.324721e+09,NaN,9th & 11th ELA,t2_6i6f9,1471965288,self.Teachers,1.472499e+09,https://www.reddit.com/r/Teachers/comments/4z6...,...,NaN,NaN,NaN,NaN,2016-08-23 15:14:48+00:00,I pretended To Be a student on the first day [...,1,compound,0.9716,2016
1,1,PhantomandaRose,1.409513e+09,NaN,English 9-10,t2_i520j,1470103158,self.Teachers,1.470666e+09,https://www.reddit.com/r/Teachers/comments/4vq...,...,NaN,NaN,NaN,NaN,2016-08-02 01:59:18+00:00,Boycott Reddit Gifts for Teachers + DonorsChoo...,2,compound,0.8845,2016
2,2,supertizer,1.285323e+09,NaN,NaN,t2_4dd3n,1480861135,supertizer.org,NaN,https://www.reddit.com/r/Teachers/comments/5gf...,...,NaN,NaN,NaN,NaN,2016-12-04 14:18:55+00:00,This is the pop quiz I give students after tea...,3,compound,0.0000,2016
3,3,supertizer,1.285323e+09,NaN,NaN,t2_4dd3n,1480263081,supertizer.org,NaN,https://www.reddit.com/r/Teachers/comments/5f5...,...,NaN,NaN,NaN,NaN,2016-11-27 16:11:21+00:00,The girls are bigger.nan,4,compound,0.0000,2016
4,4,Namnagort,1.371766e+09,NaN,NaN,t2_c3vjf,1482753867,self.Teachers,NaN,https://www.reddit.com/r/Teachers/comments/5kd...,...,NaN,NaN,NaN,NaN,2016-12-26 12:04:27+00:00,The real ten tips to avoiding burnouts1. Drink...,5,compound,-0.6664,2016


In [13]:
top_2016_final = top_2016_final[['author', 'created_utc', 'date', 'combined', 'row_id', 'sentiment_score', 'year']]

In [14]:
top_2016_final.head()

,author,created_utc,date,combined,row_id,sentiment_score,year
0,Augustane,1471965288,2016-08-23 15:14:48+00:00,I pretended To Be a student on the first day [...,1,0.9716,2016
1,PhantomandaRose,1470103158,2016-08-02 01:59:18+00:00,Boycott Reddit Gifts for Teachers + DonorsChoo...,2,0.8845,2016
2,supertizer,1480861135,2016-12-04 14:18:55+00:00,This is the pop quiz I give students after tea...,3,0.0000,2016
3,supertizer,1480263081,2016-11-27 16:11:21+00:00,The girls are bigger.nan,4,0.0000,2016
4,Namnagort,1482753867,2016-12-26 12:04:27+00:00,The real ten tips to avoiding burnouts1. Drink...,5,-0.6664,2016


In [15]:
top_2016_final.to_csv('top_2016_final.csv')